<a href="https://colab.research.google.com/github/tanaymukherjee/Deep-Learning-with-PyTorch/blob/master/04_Feedforward_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feedforward Neural Network

## Import packages and libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Device configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyper-parameters

In [3]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## MNIST dataset

In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


## Data loader

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Establishing Neural Network

In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

## Loss and optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

## Train the model

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3554
Epoch [1/5], Step [200/600], Loss: 0.1757
Epoch [1/5], Step [300/600], Loss: 0.2961
Epoch [1/5], Step [400/600], Loss: 0.2593
Epoch [1/5], Step [500/600], Loss: 0.1330
Epoch [1/5], Step [600/600], Loss: 0.1831
Epoch [2/5], Step [100/600], Loss: 0.2035
Epoch [2/5], Step [200/600], Loss: 0.1072
Epoch [2/5], Step [300/600], Loss: 0.0547
Epoch [2/5], Step [400/600], Loss: 0.0965
Epoch [2/5], Step [500/600], Loss: 0.0951
Epoch [2/5], Step [600/600], Loss: 0.1986
Epoch [3/5], Step [100/600], Loss: 0.0182
Epoch [3/5], Step [200/600], Loss: 0.0581
Epoch [3/5], Step [300/600], Loss: 0.2835
Epoch [3/5], Step [400/600], Loss: 0.0933
Epoch [3/5], Step [500/600], Loss: 0.0545
Epoch [3/5], Step [600/600], Loss: 0.0895
Epoch [4/5], Step [100/600], Loss: 0.0172
Epoch [4/5], Step [200/600], Loss: 0.0773
Epoch [4/5], Step [300/600], Loss: 0.1114
Epoch [4/5], Step [400/600], Loss: 0.0269
Epoch [4/5], Step [500/600], Loss: 0.0143
Epoch [4/5], Step [600/600], Loss:

## Test the model

In [10]:
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.55 %


## Save the model checkpoint

In [11]:
torch.save(model.state_dict(), 'model.ckpt')